# Table of contents<a class="anchor" id="table"></a>

* [1 Working with RDD](#1)
* [1.1 Data Preparation and Loading](#1.1)
* [1.1.1 Creating SparkSession & SparkContext](#OneOneOne)
* [1.1.2 Read CSV files, Preprocessing, and final(formatted data) RDD for all files](#OneOneTwo)
* [1.1.2.1 Flights RDD](#1.1.2.1)
* [1.1.2.2 Airports RDD](#1.1.2.2)
* [1.1.3 Show RDD number of columns, and number of records](#1.1.3)
* [1.2 Dataset flights partitioning](#1.2)
* [1.2.1 Obtain the maximum arrival time ](#1.2.1)
* [1.2.2 Obtain the maximum minimum time ](#1.2.2)
* [1.2.3 Perform range partitioning, and display the number of records in each partition](#1.2.3)
* [1.3 Query RDD](#1.3)
* [1.3.1 Collect a total number of flights for each month for all flights](#1.3.1)
* [1.3.2 Collect the average delay for each month for all flights](#1.3.2)
* [2 Working with DataFrames](#2)
* [2.1 Data Preparation and Loading](#2.1)
* [2.1.1 Define DataFrames](#2.1.1)
* [2.1.2 Display the Scheme of DataFrames](#2.1.2)
* [2.2.1 January Flights Events with ANC airport](#2.2.1)
* [2.2.2 Average Arrival Delay From Origin to Destination](#2.2.2)
* [2.2.3 Join Query with Airports DataFrame](#2.2.3)
* [2.3 Analysis](#2.3.1)
* [2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights](#2.3.1)
* [2.3.2 Display mean arrival delay each month](#2.3.2)
* [2.3.3 Relationship between mean departure delay and mean arrival delay](#2.3.3)
* [3 RDDs vs DataFrame vs Spark SQL](#3)
* [3.1 RDD Operation](#3.1)
* [3.2 DataFrame Operation](#3.1)
* [3.3 Spark SQL Operation](#3.1)



# 1 Working with RDD<a class="anchor" id="1"></a>
## 1.1 Data Preparation and Loading<a class="anchor" id="1.1"></a>
### 1.1.1 Create SparkSession and SparkContext<a class="anchor" id="OneOneOne"></a>
[Back to top](#table)

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import os
from pyspark.sql import Window
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql import Row
import time
import numpy as np

# Import SparkContext 
from pyspark import SparkContext # Spark

### 1.1.2 Import CSV files and Make an RDD for all files<a class="anchor" id="OneOneTwo"></a>
[Back to top](#table)

In [2]:
# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
master = "local[*]"
# Giving the app name of Assignment 1 to be shown on the Spark cluster UI page
app_name = "Assignment 1"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

#### 1.1.2.1 Flights RDD <a class="anchor" id="1.1.2.1"></a>
[Back to top](#table)

In [3]:
# access flights file from flights.csv
flights_raw_rdd = sc.textFile('flight*.csv')
# https://sparkbyexamples.com/apache-spark-rdd/spark-read-multiple-text-files-into-a-single-rdd/#directory (reference - Read all text files matching a pattern to single RDD)

#take the array first index of array as a string
#split the string into the array using comma separator
#https://www.w3schools.com/python/ref_string_split.asp (reference)
colNameFlights = flights_raw_rdd.take(1)[0].split(",")

In [4]:
flights_int_columns = ['YEAR', 'MONTH', 'DAY','DAY_OF_WEEK','FLIGHT_NUMBER']
flights_float_columns = ['DEPARTURE_DELAY','ARRIVAL_DELAY','TAXI_OUT','ELAPSED_TIME','AIR_TIME','DISTANCE','TAXI_IN','TAXI_OUT'] 

In [5]:
# remove header RDD
header1 = flights_raw_rdd.first() #extract header
flights_rdd = flights_raw_rdd.filter(lambda row: row != header1)   #filter out header

# parsing RDD
#splitting each record in each row with coma
flights_rdd = flights_rdd.map(lambda line: line.split(','))

def cast_data(line):
    for x in range(len(line)):
        if x == 0 or x == 1 or x == 2 or x ==3 or x == 5:
            if line[x]:
                line[x] = int(line[x])
            else:
                line[x] = 0  
        if x == 11 or x == 12 or x == 15 or x == 16 or x == 17 or x == 19 or x ==22:
            if line[x]:
                line[x] = float(line[x])
            else:
                line[x] = 0
    return line
            
#casting data 
flights_rdd = flights_rdd.map(cast_data)
#mapping each row to a Row object 
flights_rdd = flights_rdd.map(lambda line: Row(YEAR=line[0], MONTH=line[1], DAY=line[2], DAY_OF_WEEK=line[3], AIRLINE=line[4], FLIGHT_NUMBER=int(line[5]), TAIL_NUMBER=line[6], ORIGIN_AIRPORT=line[7], DESTINATION_AIRPORT=line[8], SCHEDULED_DEPARTURE=line[9], DEPARTURE_TIME=line[10], DEPARTURE_DELAY=line[11], TAXI_OUT=line[12], WHEELS_OFF=line[13], SCHEDULED_TIME=line[14], ELAPSED_TIME= line[15], AIR_TIME=line[16], DISTANCE=line[17], WHEELS_ON=line[18], TAXI_IN=line[19], SCHEDULED_ARRIVAL=line[20], ARRIVAL_TIME=line[21], ARRIVAL_DELAY=line[22], DIVERTED=line[23], CANCELLED=line[24], CANCELLATION_REASON=line[25], AIR_SYSTEM_DELAY=line[26], SECURITY_DELAY=line[27], AIRLINE_DELAY=line[28], LATE_AIRCRAFT_DELAY=line[29], WEATHER_DELAY=line[30]))


# Example row object
# Row(YEAR=2015, MONTH=6, DAY=26, DAY_OF_WEEK=5, AIRLINE='EV', FLIGHT_NUMBER=4951, TAIL_NUMBER='N707EV', ORIGIN_AIRPORT='BHM', DESTINATION_AIRPORT='LGA', SCHEDULED_DEPARTURE='630', DEPARTURE_TIME='629', DEPARTURE_DELAY=-1.0, TAXI_OUT=13.0, WHEELS_OFF='642', SCHEDULED_TIME='155', ELAPSED_TIME=141.0, AIR_TIME=113.0, DISTANCE=866.0, WHEELS_ON='935', TAXI_IN=15.0, SCHEDULED_ARRIVAL='1005', ARRIVAL_TIME='950', ARRIVAL_DELAY=-15.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='', AIR_SYSTEM_DELAY='', SECURITY_DELAY='', AIRLINE_DELAY='', LATE_AIRCRAFT_DELAY='', WEATHER_DELAY='')
# https://sparkbyexamples.com/pyspark/pyspark-row-using-rdd-dataframe/ (reference - some information about row object

#### 1.1.2.2 Airports RDD <a class="anchor" id="1.1.2.2"></a>
[Back to top](#table)

In [6]:
# access airports file
airports_raw_rdd = sc.textFile('airports.csv')

colNameAirports = airports_raw_rdd.take(1)[0].split(",")

In [7]:
# remove header RDD
header2 = airports_raw_rdd.first() #extract header
airports_rdd = airports_raw_rdd.filter(lambda row: row != header2)   #filter out header

# parsing RDD
#splitting each record in each row with coma
airports_rdd = airports_rdd.map(lambda line: line.split(','))
#mapping each row to a Row object 
airports_rdd = airports_rdd.map(lambda line: Row(IATA_CODE=line[0], AIRPORT=line[1], CITY = line[2], STATE = line[3], COUNTRY=line[4], LATITUDE= line[5], LONGITUDE= line [6]))

### 1.1.3 Show RDD number of columns, number of records, and number of partitions <a class="anchor" id="1.1.3"></a>
[Back to top](#table)

In [8]:
#for flights
print("Number of columns in flights:{}".format(len(colNameFlights)))
print("Number of records in flights:{}".format(flights_rdd.count()))
print("Number of partitions in flights:{}".format(flights_rdd.getNumPartitions()))

#for airports
print("Number of columns in airports:{}".format(len(colNameAirports)))
print("Number of partitions in airports:{}".format(airports_rdd.getNumPartitions()))
print("Number of records in airports:{}".format(airports_rdd.count()))

Number of columns in flights:31
Number of records in flights:582184
Number of partitions in flights:20
Number of columns in airports:7
Number of partitions in airports:2
Number of records in airports:322


## 1.2 Dataset Partitioning <a class="anchor" id="1.2"></a>
### 1.2.1 Obtain the maximum arrival time <a class="anchor" id="1.2.1"></a>
[Back to top](#table)

In [9]:
#MAXIMUM ARRIVAL DELAY (positive value)
arrival_delay_max_row = flights_rdd.max(key = lambda x: x['ARRIVAL_DELAY'])
arrival_delay_max= arrival_delay_max_row['ARRIVAL_DELAY']

### 1.2.2 Obtain the minimum arrival time <a class="anchor" id="1.2.2"></a>
[Back to top](#table)

In [10]:
#MINIMUM ARRIVAL DELAY (could be negative value)
arrival_delay_min_row = flights_rdd.min(key = lambda x: x['ARRIVAL_DELAY'])
arrival_delay_min= arrival_delay_min_row['ARRIVAL_DELAY']

In [11]:
#Print 
print("maximum delay :", arrival_delay_max)
print("minimum delay :", arrival_delay_min)

maximum delay : 1665.0
minimum delay : -82.0


### 1.2.3 Perform range partitioning, and display the number of records in each partition <a class="anchor" id="1.2.3"></a>
[Back to top](#table)

Highly skewed using the column of 'ARRIVAL_DELAY' for partitioning. In most flights, it is not desirable if operations are required on other columns, and it would lead to starvation in workers with less data, but prolonged processing time in workers with more data. 


In [12]:
# print("The number of records in each partition are ")

# determine the bin number for a single partition
no_of_partitions = 10
bin_size = (arrival_delay_max - arrival_delay_min) / (no_of_partitions)

#create the range array
range_arr=[]
btm_value = arrival_delay_min
top_value = arrival_delay_max
for x in range(10):
    temp_arr=[]
    temp_arr.append(btm_value)
    top_value = btm_value + bin_size
    temp_arr.append(top_value)
    btm_value=top_value
    range_arr.append(temp_arr)


def range_function(key):
    for index,item in enumerate(range_arr):
        if key >=item[0] and key <=item[1]:
            return index
        
partitioned_flights_rdd = flights_rdd.map( lambda x: (x['ARRIVAL_DELAY'], 1))
partitioned_flights_rdd = partitioned_flights_rdd.partitionBy(no_of_partitions, range_function)
print(f"Total partitions: {flights_rdd.getNumPartitions()}")

# glom(): Return an RDD created by coalescing all elements within each partition into a list
partitions = partitioned_flights_rdd.glom().collect()
for index,partition in enumerate(partitions):
    print(f"Partition {index}: {len(partition)} records")

Total partitions: 20
Partition 0: 564038 records
Partition 1: 16568 records
Partition 2: 1237 records
Partition 3: 196 records
Partition 4: 70 records
Partition 5: 33 records
Partition 6: 21 records
Partition 7: 12 records
Partition 8: 4 records
Partition 9: 5 records


## 1.3 Query RDD  <a class="anchor" id="1.3"></a>
### 1.3.1 Collect a total number of flights for each month <a class="anchor" id="1.3.1"></a>
[Back to top](#table)

In [13]:
# group the values by month and get the total number of flights
flights_rdd_monthly = flights_rdd.groupBy(lambda x: x['MONTH']).map(lambda line: (line[0], len(line[1]))).collect()

# loop through the values to print out the output
for x in range(len(flights_rdd_monthly)):
    print("Flights in month " + str(flights_rdd_monthly[x][0]) + " is " + str(flights_rdd_monthly[x][1]))

Flights in month 1 is 47136
Flights in month 2 is 42798
Flights in month 3 is 50816
Flights in month 4 is 48810
Flights in month 5 is 49691
Flights in month 6 is 50256
Flights in month 7 is 52065
Flights in month 8 is 50524
Flights in month 9 is 46733
Flights in month 10 is 48680
Flights in month 11 is 46809
Flights in month 12 is 47866


### 1.3.2 Collect the average delay for each month <a class="anchor" id="1.3.2"></a>
[Back to top](#table)

In [14]:
from operator import add

# getting the sum of arrival delay for each month
flights_rdd_sum_delay = flights_rdd.map(lambda x: [x['MONTH'], x['ARRIVAL_DELAY']]).reduceByKey(add).collect()

# print out the average value 
for x in range(len(flights_rdd_monthly)):
    print("Average delay of flights in month: " + str(flights_rdd_sum_delay [x][0]) + " is " + str(flights_rdd_sum_delay[x][1]/flights_rdd_monthly[x][1]))


Average delay of flights in month: 1 is 5.652155465037339
Average delay of flights in month: 2 is 7.722627225571288
Average delay of flights in month: 3 is 4.889286838790932
Average delay of flights in month: 4 is 3.1355050194632246
Average delay of flights in month: 5 is 4.644402406874485
Average delay of flights in month: 6 is 9.534662527857371
Average delay of flights in month: 7 is 6.701373283395755
Average delay of flights in month: 8 is 4.652501781331645
Average delay of flights in month: 9 is -0.8448847709327456
Average delay of flights in month: 10 is -0.5383935907970419
Average delay of flights in month: 11 is 0.8206114208805999
Average delay of flights in month: 12 is 6.035244223457151


# 2 Working with DataFrame <a class="anchor" id="2"></a>
## 2.1. Data Preparation and Loading <a class="anchor" id="2.1"></a>
### 2.1.1 Define dataframes and loading scheme<a class="anchor" id="2.1.1"></a>
[Back to top](#table)

In [15]:
# load all flights and airports data into the dataframes
flightsDf = spark.read.csv("flight*.csv", header = True, inferSchema = True)
airportsDf = spark.read.csv("airports.csv", header = True, inferSchema = True)

### 2.1.2 Display the schema of the final two dataframes<a class="anchor" id="2.1.2"></a>
[Back to top](#table)

In [16]:
# schema of both dataframes
flightsDf.printSchema()
airportsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

## 2.2. Query Analysis <a class="anchor" id="2.2"></a>
### 2.2.1 January flight events with ANC airport <a class="anchor" id="2.2.1"></a>
[Back to top](#table)

In [17]:
#1. Display all the flight events in January 2015 with five columns (Month, Origin Airport, Destination Airport, Distance, and Arrival Delay), where the origin airport 'ANC' and name this dataframe as janFlightEventsAncDf.
# Get the records for January 2015
janFlightEventsAncDfn = flightsDf.filter(col("YEAR") == 2015).filter(col("MONTH")==1)
# Get the records for Origin ariport 'ANC'
janFlightEventsAncDfn = janFlightEventsAncDfn.filter(col("ORIGIN_AIRPORT")=="ANC")
# Select only 5 columns
janFlightEventsAncDfn = janFlightEventsAncDfn.select("MONTH", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "DISTANCE", "ARRIVAL_DELAY")
# Show the result
janFlightEventsAncDfn.show()

+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261|           -6|
|    1|   

### 2.2.2 Average Arrival Delay From Origin to Destination <a class="anchor" id="2.2.2"></a>
[Back to top](#table)

In [18]:
import pyspark.sql.functions as F
#From the query results on query no.1, please display a new query. Then please group by ‘ORIGIN_AIRPORT’ AND ‘DESTINATION_AIRPORT’. Add a new column and name it as ‘AVERAGE_DELAY’. This column value is the average from all ‘ARRIVAL_DELAY’ values. Then sort it based on ‘AVERAGE_DELAY’. Please name this dataframe as janFlightEventsAncAvgDf.
# groupping by the origin airport and destination airport
janFlightEventsAncAvgDf = janFlightEventsAncDfn.groupby('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')
# adding a new column, which contains the average delay
janFlightEventsAncAvgDf = janFlightEventsAncAvgDf.agg(F.avg("ARRIVAL_DELAY").alias("AVERAGE_DELAY"))
#sort based on avg_delay
janFlightEventsAncAvgDf= janFlightEventsAncAvgDf.sort("AVERAGE_DELAY")
#show the result
janFlightEventsAncAvgDf.show(truncate= False)

+--------------+-------------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVERAGE_DELAY      |
+--------------+-------------------+-------------------+
|ANC           |ADK                |-27.0              |
|ANC           |HNL                |-20.0              |
|ANC           |MSP                |-19.25             |
|ANC           |BET                |-9.090909090909092 |
|ANC           |SEA                |-6.490196078431373 |
|ANC           |BRW                |-4.333333333333333 |
|ANC           |OME                |-3.0               |
|ANC           |ADQ                |-2.6666666666666665|
|ANC           |CDV                |1.0                |
|ANC           |OTZ                |1.25               |
|ANC           |PHX                |2.0                |
|ANC           |DEN                |3.3333333333333335 |
|ANC           |PDX                |3.5                |
|ANC           |JNU                |5.0                |
|ANC           |LAS            

### 2.2.3 Join Query with Airports DataFrame <a class="anchor" id="2.2.3"></a>
[Back to top](#table)

In [19]:
#Join the results on query no. 2 janFlightEventsAncAvgDf and airportsDf using inner join operation. Name this dataframe as joinedSqlDf, and display it
# joining the airports df anf the df determined in 2.2.2 based on the origin/destination airport code
joinedSqlDf = janFlightEventsAncAvgDf.join(airportsDf , (janFlightEventsAncAvgDf["ORIGIN_AIRPORT"] == airportsDf["IATA_CODE"]) | (janFlightEventsAncAvgDf["DESTINATION_AIRPORT"] == airportsDf["IATA_CODE"]), how = 'inner' )

#showing the final result
joinedSqlDf.show()

+--------------+-------------------+------------------+---------+--------------------+-------------+-----+-------+--------+----------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|     AVERAGE_DELAY|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+--------------+-------------------+------------------+---------+--------------------+-------------+-----+-------+--------+----------+
|           ANC|                BRW|-4.333333333333333|      ANC|Ted Stevens Ancho...|    Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                BRW|-4.333333333333333|      BRW|Wiley Post-Will R...|       Barrow|   AK|    USA|71.28545|  -156.766|
|           ANC|                ADK|             -27.0|      ADK|        Adak Airport|         Adak|   AK|    USA|51.87796|-176.64603|
|           ANC|                ADK|             -27.0|      ANC|Ted Stevens Ancho...|    Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                OME|              -3.0|

## 2.3. Analysis <a class="anchor" id="2.3"></a>
### 2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights <a class="anchor" id="2.3.1"></a>
[Back to top](#table)

In [20]:
#Find the total number of flights events, total time delay and average of arrival delay for each day of week (‘DAY_OF_WEEK’) sorted by the value of NumOfFlights in descending order. This query represents the summary of all 2015 flights. Please discuss what you observe from this query results.
# group by the weekday
weekdayRelationshipDf = flightsDf.groupby('DAY_OF_WEEK')
# get all the needed data
weekdayRelationshipDf = weekdayRelationshipDf.agg(F.avg("ARRIVAL_DELAY").alias("MeanArrivalDelay"), F.sum("ARRIVAL_DELAY").alias("TotalTimeDelay"), F.count('DAY_OF_WEEK').alias('NumOfFlights'))
# descenfing order by the num of flights
weekdayRelationshipDf = weekdayRelationshipDf.sort('NumOfFlights', ascending=False)
# sort the data by Num of flights in descending order
weekdayRelationshipDf.show()

# Observation: from the obtained result it may be observed that monday and thursday are the weekdays that have the highest
# number of flights, whereas saturday and sunday, the weekend, have the lowest. It may be also noticed that the
# higher is the numer of flights a day, the higher is the total time delay and, thus, the mean arrival delay.

+-----------+------------------+--------------+------------+
|DAY_OF_WEEK|  MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----------+------------------+--------------+------------+
|          4| 5.684831897201573|        490186|       87683|
|          1| 5.883000999381335|        494478|       86317|
|          5| 4.715112525093624|        401638|       86253|
|          3|3.9745505431431147|        335150|       85607|
|          2| 4.391518272706391|        363262|       84449|
|          7| 4.299206488272548|        343498|       81422|
|          6| 1.813841449342257|        125750|       70453|
+-----------+------------------+--------------+------------+



### 2.3.2 Display mean arrival delay each month <a class="anchor" id="2.3.2"></a>
[Back to top](#table)

In [21]:
#Find the average of arrival delay, total time delay, and total number of flight events for each month (‘MONTH’) sorted by MeanArrivalDelay in ascending order (default). Please discuss what you observe from this query results.
# group by the month
monthRelationshipDf = flightsDf.groupby('MONTH')
# get all the needed data
monthRelationshipDf = monthRelationshipDf.agg(F.avg("ARRIVAL_DELAY").alias("MeanArrivalDelay"), F.sum("ARRIVAL_DELAY").alias("TotalTimeDelay"), F.count('MONTH').alias('NumOfFlights'))
# descenfing order by the num of flights
monthRelationshipDf = monthRelationshipDf.sort('NumOfFlights')
# sort the data by Num of flights in descending order
monthRelationshipDf.show()

#Observation: from the obtained result it may be observed that July is the month that has the most flights, whereas 
# february is the one with the least. The total time delay does not significantly correlate with the number of flights
# in thid case, since both july and feb has approximately the same total time delay number, with almost 10k difference 
# between the number of flights

+-----+-------------------+--------------+------------+
|MONTH|   MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----+-------------------+--------------+------------+
|    2|  8.123906203913085|        330513|       42798|
|    9|-0.8498676252179341|        -39484|       46733|
|   11| 0.8313745860658399|         38412|       46809|
|    1|  5.804357298474946|        266420|       47136|
|   12|   6.15837046195826|        288883|       47866|
|   10| -0.541989784312509|        -26209|       48680|
|    4|  3.173803944339603|        153044|       48810|
|    5| 4.7121097658084405|        230785|       49691|
|    6|  9.747630090727856|        479174|       50256|
|    8|  4.713893233866763|        235063|       50524|
|    3|  5.011173860427592|        248454|       50816|
|    7|  6.786093552465234|        348907|       52065|
+-----+-------------------+--------------+------------+



### 2.3.3 Relationship between mean departure delay and mean arrival delay <a class="anchor" id="2.3.3"></a>
[Back to top](#table)

In [22]:
#Display the mean departure delay (MeanDeptDelay) and mean arrival delay (MeanArrivalDelay) for each month (‘MONTH’) sorted by MeanDeptDelay in descending order. Please discuss the relationship between two columns: Mean Departure Delay and Mean Arrival Delay.
# group by the weekday
deptRelationshipDf = flightsDf.groupby('MONTH')
# get all the needed data
deptRelationshipDf = deptRelationshipDf.agg(F.avg("DEPARTURE_DELAY").alias("MeanDeptDelay"), F.avg("ARRIVAL_DELAY").alias("MeanArrivalDelay"), F.sum("ARRIVAL_DELAY").alias("TotalTimeDelay"), F.count('MONTH').alias('NumOfFlights'))
# descending order by the num of flights
deptRelationshipDf = deptRelationshipDf.sort('MeanDeptDelay', ascending=False)
# sort the data by Num of flights in descending order
deptRelationshipDf.show()

#Observation: comparing the MeanDeptDelay and MeanArrivalDelay values, it may be observed that the higher the MeanDeptDelay is
# the higher the MeanArrivalDelay is as well, excluding some exceptions.

+-----+------------------+-------------------+--------------+------------+
|MONTH|     MeanDeptDelay|   MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----+------------------+-------------------+--------------+------------+
|    6|  13.9730063585922|  9.747630090727856|        479174|       50256|
|   12|11.821651454043728|   6.15837046195826|        288883|       47866|
|    7|11.708608758020432|  6.786093552465234|        348907|       52065|
|    2|11.620796080832823|  8.123906203913085|        330513|       42798|
|    8|10.086906141367324|  4.713893233866763|        235063|       50524|
|    1|  9.75401499511029|  5.804357298474946|        266420|       47136|
|    3| 9.718308159530178|  5.011173860427592|        248454|       50816|
|    5| 9.550310180006102| 4.7121097658084405|        230785|       49691|
|    4| 7.737554783759199|  3.173803944339603|        153044|       48810|
|   11| 6.630585898709037| 0.8313745860658399|         38412|       46809|
|   10| 5.243436261558784

# 3 RDDs vs DataFrame vs Spark SQL <a class="anchor" id="3"></a>


Implement the following queries using RDDs, DataFrames and SparkSQL separately. Log the time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the computational performance difference of these 3 approaches.

<strong>Find the MONTH and DAY_OF_WEEK, number of flights, and average delay where TAIL_NUMBER = ‘N407AS’. Note number of flights and average delay should be aggregated separately. The number of flights and average delay should be grouped by both MONTH and DAYS_OF_WEEK.</strong>

## 3.1 RDD Operation<a class="anchor" id="3.1"></a>
[Back to top](#table)

In [23]:
%%time
#Find and display the MONTH and DAY_OF_WEEK, number of flights, average departure delay, and average arrival delay, where TAIL_NUMBER = ‘N407AS’. Note number of flights, average departure delay, and average arrival delay should be aggregated separately. The query should be grouped by MONTH, DAY_OF_WEEK, and TAIL_NUMBER.

#get data only for tail number = ‘N407AS’
rdd_data = flights_rdd.filter(lambda line: line['TAIL_NUMBER']=="N407AS")


# group by the weekday, month and tail number and get the total number of flights per month + weekday
rdd_data_total_flights = rdd_data.groupBy(lambda x: (x['MONTH'], x['DAY_OF_WEEK'], x['TAIL_NUMBER'])).map(lambda line: (line[0], len(line[1]))).collect()
# get the average arrival delay for each month + weekday
rdd_data_arrival= rdd_data.map(lambda x: [(x['MONTH'], x['DAY_OF_WEEK'], x['TAIL_NUMBER']), x['ARRIVAL_DELAY']]).reduceByKey(add).collect()
# get the average departure delay for each month + weekday
rdd_data_dept = rdd_data.map(lambda x: [(x['MONTH'], x['DAY_OF_WEEK'], x['TAIL_NUMBER']), x['DEPARTURE_DELAY']]).reduceByKey(add).collect()

# print out the data
print("|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumOfFlights|MeanDeptDelay|MeanArrivalDelay ")
for x in range(len(rdd_data_total_flights)):
    print("   " + str(rdd_data_total_flights[x][0][0])+"|          "+str(rdd_data_total_flights[x][0][1])+"|     "+str(rdd_data_arrival[x][0][2])+"|           "+str(rdd_data_total_flights[x][1])+"|        "+str(rdd_data_dept[x][1]/rdd_data_total_flights[x][1])+"|         "+str(rdd_data_arrival[x][1]/rdd_data_total_flights[x][1]))

|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumOfFlights|MeanDeptDelay|MeanArrivalDelay 
   3|          3|     N407AS|           1|        28.0|         3.0
   9|          5|     N407AS|           1|        4.0|         5.0
   12|          2|     N407AS|           2|        1.0|         -11.5
   1|          5|     N407AS|           2|        -6.0|         -21.0
   6|          4|     N407AS|           1|        -6.0|         -15.0
   1|          1|     N407AS|           1|        4.0|         -6.0
   6|          1|     N407AS|           4|        4.5|         7.0
   2|          5|     N407AS|           1|        -11.0|         -31.0
   11|          5|     N407AS|           1|        -4.0|         12.0
   11|          1|     N407AS|           1|        57.0|         35.0
   8|          5|     N407AS|           3|        1.6666666666666667|         -10.0
   5|          6|     N407AS|           2|        0.5|         -3.0
   11|          4|     N407AS|           2|        -7.5|         -1.0
   10|     

## 3.2 DataFrame Operation<a class="anchor" id="3.2"></a>
[Back to top](#table)

In [24]:
%%time
# https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/ (reference - GroupBy examples for dataframe)

#get data only for tail number = ‘N407AS’
df_data = flightsDf.filter((F.col("TAIL_NUMBER") == "N407AS"))

# group by the weekday, month and tail number
df_data = df_data.groupby('MONTH', 'DAY_OF_WEEK', 'TAIL_NUMBER')

# get all the needed data
df_data = df_data.agg( F.count('MONTH').alias('NumOfFlights'), F.avg("DEPARTURE_DELAY").alias("MeanDeptDelay"), F.avg("ARRIVAL_DELAY").alias("MeanArrivalDelay"))

# represent the data sorting by month and weekday
df_data = df_data.sort('MONTH', 'DAY_OF_WEEK')
df_data.show()

+-----+-----------+-----------+------------+-----------------+-----------------+
|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumOfFlights|    MeanDeptDelay| MeanArrivalDelay|
+-----+-----------+-----------+------------+-----------------+-----------------+
|    1|          1|     N407AS|           1|              4.0|             -6.0|
|    1|          2|     N407AS|           2|             12.5|             17.5|
|    1|          3|     N407AS|           1|             -7.0|            -27.0|
|    1|          5|     N407AS|           2|             -6.0|            -21.0|
|    1|          6|     N407AS|           3|8.666666666666666|4.333333333333333|
|    2|          1|     N407AS|           2|             -4.0|             -2.5|
|    2|          2|     N407AS|           2|             -3.5|             -9.5|
|    2|          3|     N407AS|           2|            -12.5|            -11.5|
|    2|          4|     N407AS|           2|             -8.5|            -11.0|
|    2|          5|     N407

## 3.3 Spark SQL OPERATION<a class="anchor" id="3.3"></a>
[Back to top](#table)

In [25]:
%%time
# Create Views from Dataframes
flightsDf.createOrReplaceTempView("sql_flights")

sql_data= spark.sql('''
  SELECT MONTH, DAY_OF_WEEK, TAIL_NUMBER, count(MONTH) as NumOfFlights , AVG(DEPARTURE_DELAY) as MeanDeptDelay, AVG(ARRIVAL_DELAY) as MeanArrivalDelay
  FROM sql_flights 
  WHERE TAIL_NUMBER = 'N407AS'
  GROUP BY MONTH, DAY_OF_WEEK, TAIL_NUMBER
  ORDER BY MONTH, DAY_OF_WEEK
''')

#represent the data
sql_data.show()

+-----+-----------+-----------+------------+-----------------+-----------------+
|MONTH|DAY_OF_WEEK|TAIL_NUMBER|NumOfFlights|    MeanDeptDelay| MeanArrivalDelay|
+-----+-----------+-----------+------------+-----------------+-----------------+
|    1|          1|     N407AS|           1|              4.0|             -6.0|
|    1|          2|     N407AS|           2|             12.5|             17.5|
|    1|          3|     N407AS|           1|             -7.0|            -27.0|
|    1|          5|     N407AS|           2|             -6.0|            -21.0|
|    1|          6|     N407AS|           3|8.666666666666666|4.333333333333333|
|    2|          1|     N407AS|           2|             -4.0|             -2.5|
|    2|          2|     N407AS|           2|             -3.5|             -9.5|
|    2|          3|     N407AS|           2|            -12.5|            -11.5|
|    2|          4|     N407AS|           2|             -8.5|            -11.0|
|    2|          5|     N407

The computational time of the Spark SQL Operation (2.21ms) is significantly lower than the one of DataFrame Operation (6.97ms) 
and the one of the RDD operation (91.5ms). The DataFrame is faster than the RDD operations, since it provides an API to perform 
aggregation operations. Moreover, DataFrame has an inbuilt optimization, whereas the RDD does not have one. 
